In [1]:
import random
import torch
import pickle

import numpy as np
import torch.nn as nn
import torch.optim as optim

from pathlib import Path

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from ds.wesad.datasets import (
    get_dataset,
    get_train_test_dataset,
    subjects_data,
)
from conv_nets.wesad_tries.net_up_down_coder3_3 import (
    NetUpDownCoder3_3 as Model,
)

In [2]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.random.manual_seed(seed)

In [3]:
base_path = Path("/home/dmo/Documents/human_func_state/human_func_state")
epoch_count = 300
optim_lr = 1e-5
betas = "default"  # (0.85, 0.995) # Adam only
each_user_rate_history = True
ds_step_size = 5
train_batch_size = 8
test_batch_size = 1
device = "cuda"

In [4]:
ds_all_train, ds_all_test = get_train_test_dataset(step=ds_step_size)

In [5]:
dl_train = DataLoader(
    ds_all_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [6]:
dl_test = DataLoader(
    ds_all_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [9]:
net = Model().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.ASGD(net.parameters())

In [10]:
mod_name = (
    f"{net.__class__.__name__}"
    f"_{optimizer.__class__.__name__}"
    f"_lr_{optim_lr}"
)

write_path = base_path.joinpath("models_dumps", "wesad", "steps", mod_name)
writer = SummaryWriter(log_dir=write_path / "hist")
dump_name = (write_path / f"{mod_name}_last").with_suffix(".pkl")
best_name = (write_path / f"{mod_name}_best").with_suffix(".pkl")

In [11]:
subjects_test_datasets = {
    subject_id: get_dataset(ds_type="test", step=ds_step_size)
    for subject_id in subjects_data
}

In [12]:
def val(model: Model, dl):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl):
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            outputs = model(inputs).cpu()
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [13]:
def common_validate(model: Model):
    subject_accuracies = None
    if each_user_rate_history:
        subject_accuracies = {}
        for subject_id, subject_ds_test in subjects_test_datasets.items():
            subject_dl_test = DataLoader(
                subject_ds_test,
                batch_size=1,
                shuffle=True,
                num_workers=1,
                pin_memory=False,
                drop_last=True,
            )
            accuracy = val(model, subject_dl_test)
            subject_accuracies[subject_id] = accuracy / len(subject_ds_test)
    common_accuracy = val(model, dl_test) / len(ds_all_test)
    return common_accuracy, subject_accuracies

In [14]:
def save_data(path, net_state_dict, epoch, rate, rate_subject=None):
    with open(path, "wb") as f:
        pickle.dump(
            {
                "net_state_dict": net_state_dict,
                "current_epoch": epoch,
                "rate": rate,
                "rate_subject": rate_subject,
                "optimizer": optimizer.__class__.__name__,
                "optimizer_params": optimizer.param_groups,
                "train_batch_size": train_batch_size,
                "test_batch_size": test_batch_size,
                "device": device,
            },
            f,
        )

In [15]:
def train(
    model,
    epoch_count=50,
    print_step=50,
    start_epoch=0,
    min_loss=(np.inf, 0),
    best_rate=(0.0, 0),
    worst_rate=(1.0, 0),
):
    for epoch in range(
        start_epoch, epoch_count
    ):  # loop over the dataset multiple times
        epoch_loss = 0
        running_loss = 0.0
        model.train()
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.cpu(), labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step - 1:
                mean_loss = running_loss / print_step
                if mean_loss < min_loss[0]:
                    min_loss = (mean_loss, (epoch, i))
                print(f"[{epoch:3d}, {i:4d}] loss: {mean_loss:.3f}")
                epoch_loss += running_loss
                running_loss = 0.0

        epoch_loss += running_loss
        model.eval()
        common_acc, subjects_accs = common_validate(model)
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", common_acc, epoch)
        if each_user_rate_history:
            rates = dict(
                zip(
                    map(
                        lambda _id: f"subject_{_id}",
                        map(str, subjects_accs.keys()),
                    ),
                    subjects_accs.values(),
                )
            )
            writer.add_scalars("Accuracy_subjects/train", rates, epoch)
        if epoch % 20 == 0:
            path = (
                write_path / f"{mod_name}_epoch_{str(epoch).zfill(3)}"
            ).with_suffix(".pkl")
            save_data(path, net.state_dict(), epoch, common_acc)
        if common_acc > best_rate[0]:
            best_rate = (common_acc, epoch)
            save_data(best_name, net.state_dict(), epoch, common_acc)
        if common_acc < worst_rate[0]:
            worst_rate = (common_acc, epoch)
        save_data(dump_name, net.state_dict(), epoch, common_acc)

        print(
            f"[{epoch:3d}] rate: {common_acc:.4f}; {best_rate = }, {worst_rate = }"
        )
    print("Finished Training. Min_loss:", min_loss)
    return worst_rate, best_rate, min_loss

In [16]:
train(net, epoch_count=epoch_count)

[  0,   49] loss: 0.571
[  0,   99] loss: 0.566
[  0,  149] loss: 0.555
[  0,  199] loss: 0.527
[  0,  249] loss: 0.532
[  0,  299] loss: 0.523
[  0,  349] loss: 0.523
[  0,  399] loss: 0.520
[  0,  449] loss: 0.492
[  0,  499] loss: 0.495
[  0,  549] loss: 0.528
[  0,  599] loss: 0.516
[  0,  649] loss: 0.502
[  0,  699] loss: 0.505
[  0] rate: 0.8038; best_rate = (tensor(0.8038), 0), worst_rate = (tensor(0.8038), 0)
[  1,   49] loss: 0.482
[  1,   99] loss: 0.470
[  1,  149] loss: 0.489
[  1,  199] loss: 0.501
[  1,  249] loss: 0.487
[  1,  299] loss: 0.476
[  1,  349] loss: 0.493
[  1,  399] loss: 0.464
[  1,  449] loss: 0.490
[  1,  499] loss: 0.504
[  1,  549] loss: 0.495
[  1,  599] loss: 0.498
[  1,  649] loss: 0.513
[  1,  699] loss: 0.455
[  1] rate: 0.8563; best_rate = (tensor(0.8563), 1), worst_rate = (tensor(0.8038), 0)
[  2,   49] loss: 0.492
[  2,   99] loss: 0.464
[  2,  149] loss: 0.472
[  2,  199] loss: 0.468
[  2,  249] loss: 0.502
[  2,  299] loss: 0.494
[  2,  349] 

((tensor(0.8038), 0), (tensor(0.8942), 148), (0.32732850313186646, (255, 349)))